In [ ]:
import os, pickle, sqlite3, copy
from scipy import stats


In [ ]:
directory = os.path.dirname(os.path.abspath(filepath))
with open(filepath, 'rb') as f:
    data = pickle.load(f)
timestamps = []
for x in range(len(data)):
    for y in data[x].keys():
        timestamps.append(y)
db = sqlite3.Connection('db.sqlite')
curs = db.cursor()
tables = curs.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall()[2:]
tables = [x[0] for x in tables]
for x in tables:
    start_time = curs.execute('SELECT MIN(Timestamp) FROM ' + x + ';').fetchall()[0][0]
    if start_time > timestamps[0] and start_time < timestamps[-1]:
        table = x
        break
try:
    print('Using table: ' + table)
except NameError:
    print('Could not find a table with timestamps matching EEG data.')
    table = None
    curs.close()
    db.close()

os.chdir(os.path.dirname(filepath))
deltas = []
thetas = []
alphas = []
betas = []
gammas = []
for x in range(len(data)):
    for y in data[x].values():
        for z in range(len(y[0])):
            if z == 0:
                deltas.append(y[0][z])
            elif z == 1:
                thetas.append(y[0][z])
            elif z == 2:
                alphas.append(y[0][z])
            elif z == 3:
                betas.append(y[0][z])
            elif z == 4:
                gammas.append(y[0][z])
smooth_deltas = []
smooth_thetas = []
smooth_alphas = []
smooth_betas = []
smooth_gammas = []
for x in range(len(data)):
    for y in data[x].values():
        for z in range(len(y[1])):
            if z == 0:
                smooth_deltas.append(y[1][z])
            elif z == 1:
                smooth_thetas.append(y[1][z])
            elif z == 2:
                smooth_alphas.append(y[1][z])
            elif z == 3:
                smooth_betas.append(y[1][z])
            elif z == 4:
                smooth_gammas.append(y[1][z])
# Alpha Protocol:
# Simple redout of alpha power, divided by delta waves in order to rule out noise
alpha_over_delta = []
# Beta Protocol:
# Beta waves have been used as a measure of mental activity and concentration
# This beta over theta ratio is commonly used as neurofeedback for ADHD
beta_over_theta = []
# Alpha/Theta Protocol:
# This is another popular neurofeedback metric for stress reduction
# Higher theta over alpha is supposedly associated with reduced anxiety
theta_over_alpha = []
# Gamma/Beta:
# I suspect this may indicate when a person's intuition is at its strongest.
# Gamma waves have been associated with psi phenomena by multiple other researchers.
# Beta brainwaves are associated with analytical thinking, which is widely believed to inhibit psi.
gamma_over_beta = []
# here we go!
for x in range(len(smooth_deltas)):
    alpha_over_delta.append(smooth_alphas[x] / smooth_deltas[x])
    beta_over_theta.append(smooth_betas[x] / smooth_thetas[x])
    theta_over_alpha.append(smooth_thetas[x] / smooth_alphas[x])
    gamma_over_beta.append(smooth_gammas[x] / smooth_betas[x])
if table is not None:
    corrects = curs.execute('select timestamp from ' + table + ' where Correct=1;').fetchall()
    incorrects = curs.execute('select timestamp from ' + table + ' where Correct=0;').fetchall()
    stamps = curs.execute('select timestamp from ' + table + ' where Correct=0 or Correct=1;').fetchall()
else:
    stamps, corrects, incorrects = None, None, None

In [ ]:
avg_deltas, avg_thetas, avg_alphas, avg_betas, avg_gammas = [], [], [], [], []
avg_smooth_deltas, avg_smooth_thetas, avg_smooth_alphas, avg_smooth_betas, avg_smooth_gammas = [], [], [], [], []
avg_alpha_over_delta, avg_beta_over_theta, avg_theta_over_alpha, avg_gamma_over_beta = [], [], [], []

disposable = copy.deepcopy(data)
t = stamps.pop(0)[0]
raws, avgs, ratios = [], [], []
for x in range(len(disposable)):
    if disposable[x].popitem()[0] >= t:
        # stuff
        delts, thets, alphs, bets, gamms = [], [], [], [], []
        for y in range(len(raws)):
            delts.append(raws[x][0])
            thets.append(raws[x][1])
            alphs.append(raws[x][2])
            bets.append(raws[x][3])
            gamms.append(raws[x][4])
        avg_deltas.append(stats.mean(delts))
        avg_thetas.append(stats.mean(thets))
        avg_alphas.append(stats.mean(alphs))
        avg_betas.append(stats.mean(bets))
        avg_gammas.append(stats.mean(gamms))
        #
        s_delts, s_thets, s_alphs, s_bets, s_gamms = [], [], [], [], []
        for y in range(len(avgs)):
            s_delts.append(avgs[x][0])
            s_thets.append(avgs[x][1])
            s_alphs.append(avgs[x][2])
            s_bets.append(avgs[x][3])
            s_gamms.append(avgs[x][4])
        avg_smooth_deltas.append(stats.mean(s_delts))
        avg_smooth_thetas.append(stats.mean(s_thets))
        avg_smooth_alphas.append(stats.mean(s_alphs))
        avg_smooth_betas.append(stats.mean(s_bets))
        avg_smooth_gammas.append(stats.mean(s_gamms))
        #
        alph_delt, bet_thet, thet_alph, gamm_bet = [], [], [], []
        for y in range(len(ratios)):
            alph_delt.append(ratios[y][0])
            bet_thet.append(ratios[y][1])
            thet_alph.append(ratios[y][2])
            gamm_bet.append(ratios[y][3])
        avg_alpha_over_delta.append(stats.mean(alph_delt))
        avg_beta_over_theta.append(stats.mean(bet_thet))
        avg_theta_over_alpha.append(stats.mean(thet_alph))
        avg_gamma_over_beta.append(stats.mean(gamm_bet))
        #
        raws, avgs, ratios = [], [], []
        try:
            t = stamps.pop(0)[0]
            continue
        except IndexError:
            break
    raws.append([
        deltas[x],
        thetas[x],
        alphas[x],
        betas[x],
        gammas[x]
    ])
    avgs.append([
        smooth_deltas[x],
        smooth_thetas[x],
        smooth_alphas[x],
        smooth_betas[x],
        smooth_gammas[x]
    ])
    ratios.append([
        alpha_over_delta[x],
        beta_over_theta[x],
        theta_over_alpha[x],
        gamma_over_beta[x]
    ])